In [29]:
from astropy import units as u

from poliastro.bodies import Earth, Mars, Sun
from poliastro.twobody import Orbit

# Needed to handle TLE into Poliastro's Orbit
from tletools import TLE

# Needed for defining manouvers
from poliastro.maneuver import Maneuver

from perylune.orbit_tools import *

In [30]:
# STEP 1: LAUNCH
# Let's use ANDESITE sat as an example. It was launched as shareride by RocketLab's Electron on
# the latest rideshare launch on 2020 June 13. NORAD ID is 45726. 
# TLE elements taken from n2yo website, details https://www.n2yo.com/satellite/?s=45726

tle_text = """GDASAT-1
1 45726U 20037D   20278.45278018  .00000608  00000-0  65390-4 0  9991
2 45726  97.7132  96.2906 0012962 283.6573  76.3213 14.92011802 14275"""
tle_lines = tle_text.strip().splitlines()
tle1 = TLE.from_lines(*tle_lines)
orb1 = tle1.to_orbit()
#orb1.plot(label="Init")

print_orb(orb1)

6961 x 6979 km x 97.7 deg (GCRS) orbit around Earth (♁) at epoch 2020-10-04T10:52:00.207552000 (UTC)
𝑎=6969.80km, b=6969.79km, e=0.00, i=97.71deg raan(Ω)=96.29deg argp(𝜔)=283.66deg nu(𝜈)=76.47deg


In [31]:
# Let's get the TLE orbital data for the target satellite. As an example, let's pick a dead NOAA-17 sat.
# Its TLE data can be obtained from many places, such as celestrak or n2yo (https://www.n2yo.com/satellite/?s=27453)
tle_text = """NOAA-17
1 27453U 02032A   20263.80942421 -.00000011 +00000-0 +13583-4 0  9998
2 27453 098.5909 208.3215 0011096 327.5463 032.5033 14.25072668948324"""
tle_lines = tle_text.strip().splitlines()

tle = TLE.from_lines(*tle_lines)

orb_target = tle.to_orbit()

# orb_target.plot() # Let's not plot it yet
print_orb(orb_target)

7178 x 7194 km x 98.6 deg (GCRS) orbit around Earth (♁) at epoch 2020-09-19T19:25:34.251744000 (UTC)
𝑎=7186.39km, b=7186.38km, e=0.00, i=98.59deg raan(Ω)=208.32deg argp(𝜔)=327.55deg nu(𝜈)=32.57deg


In [32]:
# TODO1: write orbit presentation function (print_orb(o: Orbit))
# TODO2: write orbit comparison function (compare_orb(o1: Orbit, o2: Orbit))
# TODO3: Calculate maneuvers:
# - inclination change
# - Hohmann transfer (this is bullshit. Hohmann is between circular orbits)

In [58]:
from poliastro.twobody import thrust
from poliastro.twobody.propagation import cowell

f = 1 # km / s2

k = Earth.k.to(u.km ** 3 / u.s ** 2).value

a_0, _, inc_0, _, _, _ = orb1.classical()
a_f, _, inc_f, _, _, _ = orb_target.classical()

a_0 = a_0.value
a_f = a_f.value
inc_0 = inc_0.value
inc_f = inc_f.value

print("a, inc change: a_0=%f a_f =>%f, inc_0=%f => inc_f=%f" % (a_0, a_f, inc_0, inc_f))

a_d, delta_v, t_f = thrust.change_a_inc(k, a_0, a_f, inc_0, inc_f, f)
print(a_d)
print(delta_v)
print(t_f)

orb2 = orb1.propagate(t_f * u.s, method=cowell, ad=a_d, rtol=1e-0)

print("orb1=")
print_orb(orb1)
print("orb2=")
print_orb(orb2)
print("orb_target=")
print_orb(orb_target)



a, inc change: a_0=6969.800314 a_f =>7186.385343, inc_0=97.713200 => inc_f=98.590900
<function change_a_inc.<locals>.a_d at 0x7f9e5b1d3a60>
9.547198021519023
9.547198021519023
orb1=
6961 x 6979 km x 97.7 deg (GCRS) orbit around Earth (♁) at epoch 2020-10-04T10:52:00.207552000 (UTC)
𝑎=6969.80km, b=6969.79km, e=0.00, i=97.71deg raan(Ω)=96.29deg argp(𝜔)=283.66deg nu(𝜈)=76.47deg
orb2=
6966 x 7414 km x 40.7 deg (GCRS) orbit around Earth (♁) at epoch 2020-10-04T10:52:09.754750022 (UTC)
𝑎=7189.95km, b=7186.45km, e=0.03, i=40.69deg raan(Ω)=95.65deg argp(𝜔)=353.11deg nu(𝜈)=8.02deg
orb_target=
7178 x 7194 km x 98.6 deg (GCRS) orbit around Earth (♁) at epoch 2020-09-19T19:25:34.251744000 (UTC)
𝑎=7186.39km, b=7186.38km, e=0.00, i=98.59deg raan(Ω)=208.32deg argp(𝜔)=327.55deg nu(𝜈)=32.57deg


In [33]:
# This will return classical elements: Semimajor axis (𝑎), eccentricity (e), inclination (i), RAAN (Ω) - right ascension of the ascending node, 
# argument of perigeum (𝜔), nu (𝜈) - true anomaly
# orb2.classical()

In [34]:
# Define a Hohmann manouver that raises an orbit to 36000km
hoh = Maneuver.hohmann(orb1, 36000 * u.km)
print("There are %s impulses needed to conduct Hohmann transfer" % len(hoh.impulses))
print(hoh.impulses[0])
print(hoh.impulses[1])
print("The total Delta-v cost is %s, the total time is %s" % (hoh.get_total_cost(), hoh.get_total_time()))

There are 2 impulses needed to conduct Hohmann transfer
(<Quantity -1227.67876068 s>, <Quantity [-166.76473414, 2156.11804082,  520.43665092] m / s>)
(<Quantity 15665.49119097 s>, <Quantity [  107.46026366, -1389.36457003,  -335.36023077] m / s>)
The total Delta-v cost is 3.6575995674724595 km / s, the total time is 14437.812430287866 s


In [35]:
# Obtain the orbit after applying Hohmann transfer.

from poliastro.plotting import OrbitPlotter2D, OrbitPlotter3D

op = OrbitPlotter2D()
orb2, orb3 = orb1.apply_maneuver(hoh, intermediate=True)
op.plot(orb1, label="NOAA-17 - LEO,Initial orbit")
op.plot(orb2, label="Transfer orbit - Hohmann transfer")
op.plot(orb3, label="Final orbit - GEO")